# Designing LLM System

Exploring how to get the inputs needed for the ML model from the Vision LLM. The goal is to have the LLM output the recipe name, ingredients, and health labels, but we may be able to get the recipe name and health label from the ingredients list with a regular LLM, thus reducing cost as the Vision LLM is more expensive.

In [1]:
import os

# Go to root directory to import relevant files
os.chdir("../..")

## Checking Input Format

We can directly see what kind of input we will need in the feature engineering step from the ML pipeline.

In [2]:
import pandas as pd
from ml_features.ml_calorie_estimation.src.data_ingestion.utils import create_db_config, load_config
from ml_features.ml_calorie_estimation.src.databases.manager import DatabaseManager
from ml_features.ml_calorie_estimation.src.feature_engineering.text_processing import remove_stop_words, lemmatizing, get_tfidf_splits, SVD_reduction
from ml_features.ml_calorie_estimation.src.feature_engineering.data_transformations import comma_to_bracket, replace_with_priority, get_macros
from ml_features.ml_calorie_estimation.src.databases.models.clean_data import CleanRecipe

[nltk_data] Downloading package stopwords to /home/ravib/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ravib/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/ravib/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Copy this code from the feature engineering pipeline step.

In [3]:
env = "local"

# Load data from database
config = load_config(env)
db_config = create_db_config(config.database)
db_manager = DatabaseManager(db_config)
session = db_manager.Session()
query = session.query(CleanRecipe).statement
df = pd.read_sql(query, session.bind)

# Get relevant features
ingredientLines = df['ingredientLines']
healthLabels = df['healthLabels']
nutrients = df['totalNutrients']

# Feature engineering transformation code here
ingredientLines = ingredientLines.apply(comma_to_bracket)
healthLabels = healthLabels.apply(replace_with_priority)
   
X = healthLabels + " " + df['label'] + " " + ingredientLines

In [4]:
df['ingredientLines'][0]

['1 tablespoon olive oil',
 '1 large eggplant, cut into 1-inch pieces',
 '1 large brown or yellow onion, thinly slices',
 '2 medium carrots cut into 1/2 inch pieces',
 '1 can of whole tomatoes in juices',
 '2 cloves of garlic, minced or finely chopped',
 '1 tablespoon ras el hanout (or other Moroccan spice blend)',
 '1 teaspoon cumin',
 '1/4 teaspoon hot chili pepper/cayenne pepper',
 'salt',
 'pepper',
 'cilantro']

Inspect the inputs.

In [5]:
healthLabels[0]

'Vegan'

In [6]:
df['label'][0]

'Slow Cooker Moroccan Eggplant recipes'

In [7]:
ingredientLines[0]

'1 tablespoon olive oil, 1 large eggplant (cut into 1-inch pieces), 1 large brown or yellow onion (thinly slices), 2 medium carrots cut into 1/2 inch pieces, 1 can of whole tomatoes in juices, 2 cloves of garlic (minced or finely chopped), 1 tablespoon ras el hanout (or other Moroccan spice blend), 1 teaspoon cumin, 1/4 teaspoon hot chili pepper/cayenne pepper, salt, pepper, cilantro'

In [8]:
X.iloc[0]

'Vegan Slow Cooker Moroccan Eggplant recipes 1 tablespoon olive oil, 1 large eggplant (cut into 1-inch pieces), 1 large brown or yellow onion (thinly slices), 2 medium carrots cut into 1/2 inch pieces, 1 can of whole tomatoes in juices, 2 cloves of garlic (minced or finely chopped), 1 tablespoon ras el hanout (or other Moroccan spice blend), 1 teaspoon cumin, 1/4 teaspoon hot chili pepper/cayenne pepper, salt, pepper, cilantro'

## Revising LLM Prompts to Match Input Format

Now we can make sure our LLM prompts give us outputs in the same format as the inputs above.

In [27]:
import openai
import importlib

openai_api_key = os.getenv("OPENAI_API_KEY")

[autoreload of ml_features.llm_calorie_estimation.src.extractors.base failed: Traceback (most recent call last):
  File "/home/ravib/projects/iifymate/.iifymate/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/ravib/projects/iifymate/.iifymate/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/home/ravib/projects/iifymate/.iifymate/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/ravib/projects/iifymate/.iifymate/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ravib/projects/iifymate/.iifymate/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  Fi

In [28]:
%load_ext autoreload
%autoreload 2

from ml_features.llm_calorie_estimation.src.extractors import ingredients
from ml_features.llm_calorie_estimation.prompts import vision_prompts

# Reload the module to reflect changes
importlib.reload(vision_prompts)
importlib.reload(ingredients)

# Now you can access the updated INGREDIENT_LIST_PROMPT_TEMPLATE
INGREDIENT_LIST_PROMPT_TEMPLATE = vision_prompts.INGREDIENT_LIST_PROMPT_TEMPLATE

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
sample_img_path = "notebooks/data/sample_meal_images/scrambled_eggs.jpg"

In [30]:
ingredient_extractor = ingredients.IngredientExtractor(api_key=openai_api_key)

In [31]:
ingredients_response = ingredient_extractor.extract(sample_img_path, INGREDIENT_LIST_PROMPT_TEMPLATE)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [32]:
ingredients_list = ingredients_response.ingredients
ingredients_list

['3 large eggs',
 '1 tablespoon butter',
 '1/4 cup diced ham',
 '2 tablespoons chopped chives',
 'salt',
 'pepper',
 '1 slice whole grain bread',
 '1 teaspoon butter (for spreading)']

In [21]:
from ml_features.llm_calorie_estimation.src.extractors import health_labels
from ml_features.llm_calorie_estimation.src.extractors import recipe_name
from ml_features.llm_calorie_estimation.prompts import text_prompts

# Reload the module to reflect changes
importlib.reload(text_prompts)
importlib.reload(health_labels)
importlib.reload(recipe_name)

# Now you can access the updated HEALTH_LABEL_PROMPT_TEMPLATE
HEALTH_LABEL_PROMPT_TEMPLATE = text_prompts.HEALTH_LABEL_PROMPT_TEMPLATE
RECIPE_LABEL_PROMPT_TEMPLATE = text_prompts.RECIPE_LABEL_PROMPT_TEMPLATE

In [22]:
print(HEALTH_LABEL_PROMPT_TEMPLATE)


Analyze these ingredients and determine the most appropriate single health label in JSON format.

Ingredients: {{ ingredients | tojson(indent=2) }}

Instructions:
- Choose exactly one label from: ['Vegan', 'Vegetarian', 'Pescatarian', 'Paleo', 'Red-Meat-Free', 'Mediterranean']
- Consider these rules:
    * Vegan: No animal products whatsoever
    * Vegetarian: May include dairy/eggs but no meat/fish
    * Pescatarian: Includes fish but no other meat
    * Paleo: No grains, dairy, processed foods
    * Red-Meat-Free: May include poultry/fish
    * Mediterranean: Emphasizes plant-based foods, fish, olive oil

Example input: [
        "1 tablespoon olive oil",
        "1 large eggplant, cut into 1-inch pieces",
        "1 large brown or yellow onion, thinly slices",
        "2 medium carrots cut into 1/2 inch pieces",
        "1 can of whole tomatoes in juices",
        "2 cloves of garlic, minced or finely chopped",
        "1 tablespoon ras el hanout",
        "1 teaspoon cumin",
     

In [23]:
health_label_extractor = health_labels.HealthLabelExtractor(api_key=openai_api_key)

In [24]:
sample_health_label = health_label_extractor.extract(ingredients_list, prompt=HEALTH_LABEL_PROMPT_TEMPLATE)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [25]:
sample_health_label

HealthLabelResponse(health_label='Red-Meat-Free')

In [26]:
recipe_name_extractor = recipe_name.RecipeLabelExtractor(api_key=openai_api_key)
sample_recipe_name = recipe_name_extractor.extract(ingredients_list, prompt=RECIPE_LABEL_PROMPT_TEMPLATE)
sample_recipe_name

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


RecipeLabelResponse(recipe_label='Hearty Ham and Chive Scramble')